## Setup

In [1]:
import numpy as np

from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import H, I

## Quantum Oracle

In [3]:
SEARCHED_STRING = "111"
N = len(SEARCHED_STRING)
oracle = np.zeros(shape=(2 ** N, 2 ** N))
for b in range(2 ** N):
    if np.binary_repr(b, N) == SEARCHED_STRING:
        oracle[b, b] = -1
    else:
        oracle[b, b] = 1
print(oracle)

[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]]


## Initialization

In [4]:
qvm = QVMConnection()
gr_prog = Program()

In [5]:
qubits = list(reversed(range(N)))
gr_prog.inst([I(q) for q in qubits])

## Create superposition

In [6]:
gr_prog.inst([H(q) for q in qubits])

## The Loop

In [7]:
# Define quantum oracle
ORACLE_GATE_NAME = "GROVER_ORACLE"
gr_prog.defgate(ORACLE_GATE_NAME, oracle)

# Define inversion around the mean
DIFFUSION_GATE_NAME = "DIFFUSION"
diffusion = 2.0 * np.full((2**N, 2**N), 1/(2**N)) - np.eye(2**N)
gr_prog.defgate(DIFFUSION_GATE_NAME, diffusion)

# Number of algorithm iterations
N_ITER = int(np.pi / 4 * np.sqrt(2**N))

# Loop
for i in range(N_ITER):
    
    # \psi_2^i:  Apply Quantum Oracle
    gr_prog.inst(tuple([ORACLE_GATE_NAME] + qubits))
    #print(qvm.wavefunction(gr_prog))
    
    # \psi_3^i:  Apply Inversion around the mean
    gr_prog.inst(tuple([DIFFUSION_GATE_NAME] + qubits))
    #print(qvm.wavefunction(gr_prog))

## Measurement

In [8]:
# \psi_5: Measure
for q in qubits:
    gr_prog.measure(qubit_index=q, classical_reg=q)

# Run
ret = qvm.run(gr_prog, classical_addresses=qubits)
ret_string = ''.join([str(q) for q in ret[0]])
print("The searched string is: {}".format(ret_string))
print(ret)

The searched string is: 111
[[1, 1, 1]]


## Fully Implemented Grove

In [3]:
import numpy as np
from grove.amplification.grover import Grover
from pyquil.api import QVMConnection

# Bitstring Map as an algorithm input
SEARCHED_STRING = "1011110"
N = len(SEARCHED_STRING)
mapping = {}
for b in range(2 ** N):
    pad_str = np.binary_repr(b, N)
    if pad_str == SEARCHED_STRING:
        mapping[pad_str] = -1
    else:
        mapping[pad_str] = 1

# Connection
qvm = QVMConnection()

# Run
algo = Grover()
ret_string = algo.find_bitstring(qvm, bitstring_map=mapping)    
print("The searched string is: {}".format(ret_string))

ConnectionError: HTTPSConnectionPool(host='api.rigetti.com', port=443): Max retries exceeded with url: /qvm (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001E2BB49DA58>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

### Source: http://dkopczyk.quantee.co.uk/grover-search/#easy-footnote-bottom-2-498